<a href="https://colab.research.google.com/github/Ahmethan96/Neural-netweok/blob/main/Predict%20car%20prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt

import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split

In [43]:
data_frame = pd.read_csv('car_models.csv')
data_frame.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


In [45]:
## I categorized each of my variables into numbers by using
c = data_frame.Transmission.astype("category")

# Now, I would like to know if there is a way to obtain which original value 
# corresponds to each value.
d = dict(enumerate(c.cat.categories))

print(d)


{0: 'Automatic', 1: 'Manual'}


In [46]:
a = data_frame.Fuel_Type.astype("category")
b = dict(enumerate(a.cat.categories))

print(b)


{0: 'CNG', 1: 'Diesel', 2: 'Petrol'}


In [24]:
import random 
your_name = "ibrahim ahmethan"
print(ord("i"))
print(ord(your_name[0]))
print(ord(your_name[1]))
print(ord(your_name[2])) 
print(ord(your_name[3])) # unicode code of a specified character.
print(chr(105)) ## opposite of above 
def customize_dataset(data_frame, rand_str):

  new_data = data_frame.copy(deep = True)

  # new_data = new_data.sample(int(0.95*len(new_data)), random_state= int(ord(rand_str[0])))
  new_data = new_data.sample(int(0.95*len(new_data)), random_state= (random.randint(0,100)))

  new_data.Year = new_data.Year * ord(rand_str[1]) / 100

  new_data.Selling_Price = new_data.Selling_Price * ord(rand_str[2])/100

  # if ord(rand_str[3]) % 2 == 1:
  if random.randint(1, 100)> 0:
    new_data = new_data.drop(['Car_Name'], axis= 1)
    return new_data

new_data = customize_dataset(data_frame, your_name)
new_data.head() ## I had TypeError couple of time cuz def customize_dataset returnd non for some reason


105
105
98
114
97
i


,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,1973.72,3.8190,5.59,27000,Petrol,Dealer,Manual,0
246,1971.76,4.2750,6.79,35000,Petrol,Dealer,Manual,0
46,1972.74,3.0210,4.89,64532,Petrol,Dealer,Manual,0
225,1970.78,3.0780,4.60,22517,Petrol,Dealer,Manual,0
133,1975.68,0.8208,0.95,500,Petrol,Individual,Manual,0


In [25]:
input_cols = ["Year","Present_Price","Kms_Driven","Owner"]
categorical_cols = ["Fuel_Type","Seller_Type","Transmission"]
output_cols = ["Selling_Price"]

In [ ]:
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    new_data = dataframe.copy(deep=True)
    # Convert non-numeric categorical columns to numbers
    for col in categorical_cols:
        # I categorized each of my variables into numbers by using
        # .astype('category').cat.codes
        new_data[col] = new_data[col].astype('category').cat.codes

    # Extract input & outupts as numpy arrays
    inputs_array = new_data[input_cols].to_numpy()
    targets_array = new_data[output_cols].to_numpy()
    return inputs_array, targets_array

inputs_array, targets_array = dataframe_to_arrays(new_data)
inputs_array, targets_array

In [37]:
print(inputs_array)

[[1.97372e+03 5.59000e+00 2.70000e+04 0.00000e+00]
 [1.97176e+03 6.79000e+00 3.50000e+04 0.00000e+00]
 [1.97274e+03 4.89000e+00 6.45320e+04 0.00000e+00]
 ...
 [1.96980e+03 7.50000e+00 6.12030e+04 0.00000e+00]
 [1.97470e+03 1.36000e+01 2.48000e+04 0.00000e+00]
 [1.97666e+03 1.25000e+01 9.00000e+03 0.00000e+00]]


In [48]:
a = data_frame.Car_Name.astype('category')
d = dict(enumerate(a.cat.categories))
print(d)

{0: '800', 1: 'Activa 3g', 2: 'Activa 4g', 3: 'Bajaj  ct 100', 4: 'Bajaj Avenger 150', 5: 'Bajaj Avenger 150 street', 6: 'Bajaj Avenger 220', 7: 'Bajaj Avenger 220 dtsi', 8: 'Bajaj Avenger Street 220', 9: 'Bajaj Discover 100', 10: 'Bajaj Discover 125', 11: 'Bajaj Dominar 400', 12: 'Bajaj Pulsar  NS 200', 13: 'Bajaj Pulsar 135 LS', 14: 'Bajaj Pulsar 150', 15: 'Bajaj Pulsar 220 F', 16: 'Bajaj Pulsar NS 200', 17: 'Bajaj Pulsar RS200', 18: 'Hero  CBZ Xtreme', 19: 'Hero  Ignitor Disc', 20: 'Hero Extreme', 21: 'Hero Glamour', 22: 'Hero Honda CBZ extreme', 23: 'Hero Honda Passion Pro', 24: 'Hero Hunk', 25: 'Hero Passion Pro', 26: 'Hero Passion X pro', 27: 'Hero Splender Plus', 28: 'Hero Splender iSmart', 29: 'Hero Super Splendor', 30: 'Honda Activa 125', 31: 'Honda Activa 4G', 32: 'Honda CB Hornet 160R', 33: 'Honda CB Shine', 34: 'Honda CB Trigger', 35: 'Honda CB Unicorn', 36: 'Honda CB twister', 37: 'Honda CBR 150', 38: 'Honda Dream Yuga ', 39: 'Honda Karizma', 40: 'Hyosung GT250R', 41: 'KTM

In [50]:
inputs = torch.Tensor(inputs_array)
targets = torch.Tensor(targets_array) 

In [51]:
print(len(inputs))

285


In [52]:
from tables.tests.test_suite import test
new_data = TensorDataset(inputs, targets)
train_set, test_set = random_split(new_data, [228, 57])
batch_size = 128 

train_loader = DataLoader(train_set, batch_size, shuffle= True)
valdiation = DataLoader(test, batch_size)